In [187]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
from pyspark.sql.functions import round
from pyspark.sql.types import DecimalType, DoubleType
from pyspark.sql.functions import desc, asc, exp, max, col

In [112]:
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row
df = spark.createDataFrame([
    Row(a=1, b='string1', c='string1', d='string1', e=2.00),
    Row(a=2, b='string2', c='string2', d='string2', e=3.00),
    Row(a=4, b='string3', c='string3', d='string3', e=5.00)
])
df

DataFrame[a: bigint, b: string, c: string, d: string, e: double]

In [114]:
sales = spark.read.format('csv').option('header','true').load('2021_State_Employee_Pay.csv')
sales.createOrReplaceTempView('em')
sales.show(truncate=False)

+-------------+-----------+-----------------------+-------------------------+-------------+
|Calendar_Year|Agency_Name|Position_Title         |Employee_Name            |YTD_Gross_Pay|
+-------------+-----------+-----------------------+-------------------------+-------------+
|2021         |AGRICULTURE|ACCOUNTANT             |KLEINDIENST, ANGELA F    |44054.5      |
|2021         |AGRICULTURE|ACCOUNTANT             |WOOD, KAREN M.           |39339.97     |
|2021         |AGRICULTURE|ACCOUNTS SUPERVISOR    |WALKER, JOE E.           |53821.2      |
|2021         |AGRICULTURE|ADMIN SUPPORT ASSISTANT|BICKERTON, HAILEY ANN    |29484        |
|2021         |AGRICULTURE|ADMIN SUPPORT ASSISTANT|BIRDWELL, RHIANNON       |15340.08     |
|2021         |AGRICULTURE|ADMIN SUPPORT ASSISTANT|HALL, STACY A.           |20208.3      |
|2021         |AGRICULTURE|ADMIN SUPPORT ASSISTANT|HENRY, PAMELA A          |14742        |
|2021         |AGRICULTURE|ADMIN SUPPORT ASSISTANT|JONES, MEGAN L           |155

# Total spent on employ salary

In [145]:
tot_sum = sales.agg({'YTD_Gross_Pay': 'sum'})
tot_sum = tot_sum.withColumn('Expanded', tot_sum["sum(YTD_Gross_Pay)"].cast(DecimalType(20,2)))
tot_sum.show()

+-------------------+-------------+
| sum(YTD_Gross_Pay)|     Expanded|
+-------------------+-------------+
|2.160391753169996E9|2160391753.17|
+-------------------+-------------+



# Avg employ salary

In [147]:
tot_avg = sales.agg({'YTD_Gross_Pay': 'avg'})
tot_avg = tot_avg.withColumn('Expanded', tot_avg["avg(YTD_Gross_Pay)"].cast(DecimalType(20,2)))
tot_avg.show()

+------------------+--------+
|avg(YTD_Gross_Pay)|Expanded|
+------------------+--------+
|30628.648942652526|30628.65|
+------------------+--------+



# Avg Agency salary

In [172]:
agen_avg = sales.groupBy("Agency_Name").agg({"YTD_Gross_Pay": "avg"}).sort("Agency_Name")
agen_avg = agen_avg.withColumn('Avg_per_agency', agen_avg["avg(YTD_Gross_Pay)"].cast(DecimalType(20,2)))
agen_avg.sort(desc("Avg_per_agency")).show(24, truncate=False)

+----------------------------------+------------------+--------------+
|Agency_Name                       |avg(YTD_Gross_Pay)|Avg_per_agency|
+----------------------------------+------------------+--------------+
|OFFICE OF GOVERNOR                |52052.740769230775|52052.74      |
|OFFICE OF LIEUTENANT GOVERNOR     |46640.093157894735|46640.09      |
|OFFICE OF STATE TREASURER         |40724.073571428584|40724.07      |
|JUDICIARY                         |40040.17376719145 |40040.17      |
|OFFICE OF STATE AUDITOR           |39918.378322981356|39918.38      |
|OFFICE OF ATTORNEY GENERAL        |38734.770356435656|38734.77      |
|COMMERCE AND INSURANCE            |38402.44062267659 |38402.44      |
|OFFICE OF ADMINISTRATION          |36561.482666390904|36561.48      |
|LABOR AND INDUSTRIAL RELATIONS    |36034.56099514558 |36034.56      |
|HEALTH AND SENIOR SERVICES        |34112.07559065337 |34112.08      |
|LEGISLATURE                       |34017.883493699956|34017.88      |
|ECONO

# Total Salary per agency

In [171]:
agen_tot = sales.groupBy("Agency_Name").agg({"YTD_Gross_Pay": "sum"}).sort("Agency_Name")
agen_tot = agen_tot.withColumn('Total_per_agency', agen_tot["sum(YTD_Gross_Pay)"].cast(DecimalType(20,2)))
agen_tot.sort(desc("sum(YTD_Gross_Pay)")).show(24, truncate=False)

+----------------------------------+--------------------+----------------+
|Agency_Name                       |sum(YTD_Gross_Pay)  |Total_per_agency|
+----------------------------------+--------------------+----------------+
|CORRECTIONS                       |3.822000074700007E8 |382200007.47    |
|MENTAL HEALTH                     |2.7705459687000114E8|277054596.87    |
|PUBLIC SAFETY                     |2.3506643366000026E8|235066433.66    |
|SOCIAL SERVICES                   |2.3418771013999888E8|234187710.14    |
|TRANSPORTATION                    |2.326571463500011E8 |232657146.35    |
|JUDICIARY                         |2.008815517899995E8 |200881551.79    |
|OFFICE OF ADMINISTRATION          |8.844222656999959E7 |88442226.57     |
|HEALTH AND SENIOR SERVICES        |7.883300668999992E7 |78833006.69     |
|CONSERVATION                      |6.992013162000003E7 |69920131.62     |
|ELEMENTARY AND SECONDARY EDUCATION|6.925081386999997E7 |69250813.87     |
|NATURAL RESOURCES       

# Number of employess

In [156]:
agen_employ = sales.groupBy("Agency_Name").agg({"Agency_Name": "count"}).sort("Agency_Name")
agen_employ.sort(desc("count(Agency_Name)")).show(24, truncate=False)

+----------------------------------+------------------+
|Agency_Name                       |count(Agency_Name)|
+----------------------------------+------------------+
|CORRECTIONS                       |12530             |
|MENTAL HEALTH                     |10089             |
|SOCIAL SERVICES                   |8362              |
|PUBLIC SAFETY                     |7385              |
|TRANSPORTATION                    |7170              |
|JUDICIARY                         |5017              |
|ELEMENTARY AND SECONDARY EDUCATION|2845              |
|OFFICE OF ADMINISTRATION          |2419              |
|HEALTH AND SENIOR SERVICES        |2311              |
|NATURAL RESOURCES                 |2310              |
|CONSERVATION                      |2303              |
|REVENUE                           |1929              |
|AGRICULTURE                       |1273              |
|COMMERCE AND INSURANCE            |1076              |
|LEGISLATURE                       |873         

# agency total lowest to highest 

In [159]:
lowest_em = agen_tot.sort(asc("sum(YTD_Gross_Pay)"))
lowest_em.show(24, truncate=False)

+----------------------------------+--------------------+----------------+
|Agency_Name                       |sum(YTD_Gross_Pay)  |Total_per_agency|
+----------------------------------+--------------------+----------------+
|OFFICE OF LIEUTENANT GOVERNOR     |886161.7699999999   |886161.77       |
|OFFICE OF GOVERNOR                |2030056.8900000004  |2030056.89      |
|OFFICE OF STATE TREASURER         |2280548.1200000006  |2280548.12      |
|OFFICE OF STATE AUDITOR           |6426858.909999998   |6426858.91      |
|ECONOMIC DEVELOPMENT              |6982084.72          |6982084.72      |
|OFFICE OF SECRETARY OF STATE      |9082329.97          |9082329.97      |
|HIGHER EDUCATION AND WORKFORCE DEV|1.4353807269999988E7|14353807.27     |
|AGRICULTURE                       |1.710069344E7       |17100693.44     |
|OFFICE OF ATTORNEY GENERAL        |1.9561059030000005E7|19561059.03     |
|LABOR AND INDUSTRIAL RELATIONS    |2.9692478259999957E7|29692478.26     |
|LEGISLATURE             

# agency total highest to lowest 

In [162]:
highest_em = agen_tot.sort(desc("sum(YTD_Gross_Pay)"))
highest_em.show(24, truncate=False)

+----------------------------------+--------------------+----------------+
|Agency_Name                       |sum(YTD_Gross_Pay)  |Total_per_agency|
+----------------------------------+--------------------+----------------+
|CORRECTIONS                       |3.822000074700007E8 |382200007.47    |
|MENTAL HEALTH                     |2.7705459687000114E8|277054596.87    |
|PUBLIC SAFETY                     |2.3506643366000026E8|235066433.66    |
|SOCIAL SERVICES                   |2.3418771013999888E8|234187710.14    |
|TRANSPORTATION                    |2.326571463500011E8 |232657146.35    |
|JUDICIARY                         |2.008815517899995E8 |200881551.79    |
|OFFICE OF ADMINISTRATION          |8.844222656999959E7 |88442226.57     |
|HEALTH AND SENIOR SERVICES        |7.883300668999992E7 |78833006.69     |
|CONSERVATION                      |6.992013162000003E7 |69920131.62     |
|ELEMENTARY AND SECONDARY EDUCATION|6.925081386999997E7 |69250813.87     |
|NATURAL RESOURCES       

In [192]:
sales2 = sales.withColumn("YTD_Gross_Pay",sales.YTD_Gross_Pay.cast(DoubleType()))

In [202]:
sales2.sort(desc("YTD_Gross_Pay")).show(6, truncate=False)

+-------------+-------------+---------------------+-----------------------+-------------+
|Calendar_Year|Agency_Name  |Position_Title       |Employee_Name          |YTD_Gross_Pay|
+-------------+-------------+---------------------+-----------------------+-------------+
|2021         |MENTAL HEALTH|CLIENT/PATIENT WORKER|CLIENT/PATIENT WORKERS,|982768.7     |
|2021         |MENTAL HEALTH|PSYCHIATRIST         |VANGALA, SEKHAR        |439629.33    |
|2021         |MENTAL HEALTH|PSYCHIATRIST         |LELE, MANISHA M.       |418077.8     |
|2021         |MENTAL HEALTH|PSYCHIATRIST         |YARASI, NAVEEN K.      |401102.64    |
|2021         |MENTAL HEALTH|PSYCHIATRIST         |EL-RUWIE, NEZAR A.     |388272.2     |
|2021         |MENTAL HEALTH|PSYCHIATRIST         |NEFF, STACY L.         |323526.08    |
+-------------+-------------+---------------------+-----------------------+-------------+
only showing top 6 rows



In [203]:
sales2.sort(asc("YTD_Gross_Pay")).show(105, truncate=False)

+-------------+--------------------------+------------------------------+---------------------------+-------------+
|Calendar_Year|Agency_Name               |Position_Title                |Employee_Name              |YTD_Gross_Pay|
+-------------+--------------------------+------------------------------+---------------------------+-------------+
|2021         |MENTAL HEALTH             |REGISTERED NURSE              |WHEATLEY, ELIZABETH        |-9302.84     |
|2021         |PUBLIC SAFETY             |SPECIAL ASST PROFESSIONAL     |BALQEES, RASHIDA           |-3217.5      |
|2021         |MENTAL HEALTH             |DIRECT CARE AIDE              |HOWARD, CLARISSA L         |-3120.95     |
|2021         |PUBLIC SAFETY             |SERGEANT                      |MILLER, SCOTT A            |-1020.3      |
|2021         |MENTAL HEALTH             |PSYCHIATRIC TECHNICIAN I      |FURTWANGLER, CHAES A.      |-993.51      |
|2021         |CORRECTIONS               |CORRECTIONAL OFFICER          

In [193]:
#sales.select(max(col("YTD_Gross_Pay"))).show()
#sales.groupby().max('YTD_Gross_Pay').first().asDict()['max(YTD_Gross_Pay)']
#spark.sql("SELECT MAX(e) as maxval FROM sales").first().asDict()['maxval']
#sales.groupby().max('YTD_Gross_Pay').first().asDict()['max(YTD_Gross_Pay)']
sales2.dtypes

[('Calendar_Year', 'string'),
 ('Agency_Name', 'string'),
 ('Position_Title', 'string'),
 ('Employee_Name', 'string'),
 ('YTD_Gross_Pay', 'double')]

In [184]:
highest_em.dtypes

[('Agency_Name', 'string'),
 ('sum(YTD_Gross_Pay)', 'double'),
 ('Total_per_agency', 'decimal(20,2)')]